# pytest概述
pytest是python的一种单元测试框架，与unittest测试框架类似，
但是使用起来更简洁，效率更高。

pytest的特点：
* 简单的断言
* 自动发现测试模块和测试函数
* 良好的文档

# pytest中的fixture
在写测试用例时，如果测试函数中都是没有副作用的函数，列入一些数学函数，给固定输入就可以得到固定输出，那么测试用例就会特别好写.可是没有副作用的函数比较少见．所以在测试前，一般需要做一些准备．测试结束后，做一些处理．


## pytest中的setup与teardown
  fixture不大好意会，大概是"固定装置", '测试夹具'的意思.如果将其换成一种称呼，就会比较好理解-setup与teardown,也就是在测试前后，做一些准备和清理.
  pytest中，支持``setup_*``与``teardwon_*``形式的function和method,分别在测试样例的前后调用.**有module, function,class和method四种level**，大致形式如下:
  
  ```
  def setup_module(module):
    pass

def teardown_module(module):
    pass

def setup_function(function):
    pass

def teardown_function(function):
    pass

class TestSomething:
    @classmethod
    def setup_class(cls):
        pass

    @classmethod
    def teardown_class(cls):
        pass

    def setup_method(self, method):
        pass

    def teardown_method(self, method):
        pass
  ```
  
  pytest的3.0版本以后，上面展示的``module, function, method``可以去掉，参数根据需要调整,一般不需要.但是 ``class``中的``cls``不能去掉．
  
  ``setup_module``是在同一个module的测试执行前调用一次,``teardown_module``则是在之后调用一次．``setup_function``与``teardown_function``则是在每次``test_*``形式的函数被执行的前后调用.``class与method``的机制也类似.
  
  **这种写法比较古老,是为了兼容unittest而保留的**,并非pytest推荐的写法.它存在的问题是，对需要被setup和teardown的东西，分得不够细致．假如，有一个资源－－比如一个伪造的数据库，会被三个测试function使用，而这个module有10个．按照这种写法，就不存在一个简洁的写法，令pytest仅为这3个function准备，而不影响另外的7个.
  这时候，pytest的fixture就可以大显伸手了
  

## pytest.fixture

```
import pytest
import smtplib


@pytest.fixture(scope="module")
def smtp():
    smtp = smtplib.SMTP("smtp.gmail.com", 587, timeout=5)
    yield smtp
    smtp.close()


def test_ehlo(smtp):
    response, msg = smtp.ehlo()
    assert response == 250
    assert b"smtp.gmail.com" in msg
```
  这个是一个官方样例，明确展示了fixture用法的各个细节.首先fixture作为装饰器,被它作用的function即成为一个fixture.
  
  ``scope='module'``是指定作用域．类似``setup_*``与``teardown_*``,这里的scope支持**function, class, module, session**四种,默认scope是function．新增的session扩大到整个测试，可以覆盖多个module.
  
  fixture的function名称,可以直接当作参数,传给需要使用他们的测试样例．**在使用时, ``test_*``function的参数``smtp``使用的并非是前面定义的``function def smtp()``,而是``yield后的smtp，即smtplib.SMTP``**.
  
  ``yiled smtp``而不是``return smtp``,否则之后不能调用``smtp.close()``.使用``yield``,则后面的内容就是``teardown``.这样不仅方便，而且把同一组的预备，清理写在一起，逻辑上更加紧密.
  
  最终,``test_ehlo``函数中直接声明一个参数``smtp``,就可以使用这个fixture.同一个测试function中可以声明多个这个这类的形式参数，也可以混杂其他类型的参数.如果那些没使用``smtp``这个fixture的function被单独测试，它也不会被调用.
  
  另外，在fixture中,可使用其他fixture作为形式参数，形成树状依赖.这为测试环境的准备，提供了更高的抽象层级.
  

## pytest的conftest.py

  前面有提，fixture的scope中，有session，也就是整个测试过程。 这意味着，fixture可以是全局的，供多个module使用。
 
  pytest支持在测试的路径下，使用conftest.py文件进行全局配置.
  
  ```
  tests
├── conftest.py
├── test_a.py
├── test_b.py
└── sub
    ├── __init__.py
    ├── conftest.py
    ├── test_c.py
    └── test_d.py
  ```

在以上目录结构下，顶层的conftest.py里的配置，可以给四个测试module使用。 而sub下面的conftest.py，只能给sub下面的两个module使用。 如果两个conftest.py中定义了名称相同的fixture，则可以被覆盖； 也就是说，在sub下面的module，使用的是sub下的conftest.py里的定义同名fixture。

##  内置fixture
以下命令可以列出所有可用的fixture，包括内置的、插件中的、以及当前项目定义的
```
pytest --fixtures
```

其中不乏广泛应用的内容，比如capsys和tmpdir。
```
def test_print(capsys):
    print('hello')
    out, err = capsys.readouterr()
    assert 'hello' == out

def test_path(tmpdir):
    from py._path.local import LocalPath
    assert isinstance(tmpdir, LocalPath)
    from os.path import isdir
    assert isdir(str(tmpdir))
```

capsys可以捕捉测试function的标准输出，而tmpdir则可以自动创建临时文件夹。 它们都是常用fixture，如果没有内置，恐怕所有项目都要自行实现。

## fixtrue Parametrizing
有时候，测试一个function,需要测试多种情况．而每一种测试的逻辑基本雷同，只是参数或者环境有差异．这时需要参数化的fixture来减少重复.
  
  比如，前面的``smtp``那个例子，可能需要多个邮箱来测试.
  
  ```
@pytest.fixture(params=["smtp.gmail.com", "mail.python.org"])
def smtp(request):
    smtp = smtplib.SMTP(request.param, 587, timeout=5)
    yield smtp
    print ("finalizing %s" % smtp)
    smtp.close()
  ```
  
  通过在``pytest.fixture``中，指定参数``params``,就可以利用特殊的对象``request``来引用``request.param``.使用以上的``smtp``测试样例,都会被执行两次.
  
  还有另外一种情况，直接对测试function进行参数化．
  
  ```
  def add(a, b):
    return a + b

@pytest.mark.parametrize("test_input, expected", [
    ([1, 1], 2),
    ([2, 2], 4),
    ([0, 1], 1),
])
def test_add(test_input, expected):
    assert expected == add(test_input[0], test_input[1])
  ```
  
  利用``@pytest.mark.parametrize``，可以无需没有实质意义的fixture，直接得到参数化的效果，测试多组值。
  


## 插件

pytest-cov http://note.qidong.name/2018/04/pytest-plugins/
pytest-mock http://note.qidong.name/2018/02/pytest-mock/
